In [ ]:
# !apt-get update 
# !pip3 install pybullet
# !apt-get install -y ffmpeg
# https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#

In [1]:
import pybullet as p
import time
import numpy as np
import os
import sys
from collections import namedtuple

import _utils_v2 as utils
from _utils_v2 import geom_shape, xyz_v2, s, CFG_Loader

from builder import initializer, builder, main
# from body_move import move


class daughter:
    def __init__(self, 
                 path2cfgs, 
                 VARIABLE        = 0.04,
                 fig_joint_coeff = 0.25,
                 base_body_Mass  = 200,#0.001,
                 centered        = False):
        # super(daughter, self).__init__()
        
        self.path2cfgs = path2cfgs
        self.f_start   = True
        self.VARIABLE  = VARIABLE
        self.joint_cf  = fig_joint_coeff
        self.body_Mass = base_body_Mass
        self.position  = utils.coordinates(0,0,0) if centered else utils.coordinates(0,0,1)
        self.centered  = centered 
        self.init_cfgs = initializer(self.path2cfgs, self.VARIABLE, self.joint_cf)
        
    def run_enviroment(self, mode = p.GUI):
        p.connect(mode)# Avalible p.GUI p.DIRECT
        p.createCollisionShape(p.GEOM_PLANE, planeNormal = [0,0,1])
        p.createMultiBody(0,0)
        
        self.blocks = builder(self.init_cfgs, self.position)
        self.body   = main(self.blocks, self.body_Mass)
        self.names  = [data.Name for data in self.body.cfgs if data.jntTypes==0]
        self.minmax = [data.angles for data in self.body.cfgs if data.jntTypes==0]
        self.mtrIdx = [i for i, data in enumerate(self.body.cfgs) if data.jntTypes==0]
        self.body.create_body(self.centered)
        
    def stop_enviroment(self):
        p.disconnect()   
    
    @staticmethod
    def clamp(angles):
        assert len(angles) == len(self.minmax)
        for i in range(len(angles)):
            if angles[i]  < self.minmax[i][0]:
                angles[i] = self.minmax[i][0]

            if angles[i]  > self.minmax[i][1]:
                angles[i] = self.minmax[i][1]
        return angles
    
    def reset(self):
        p.resetBasePositionAndOrientation(1, [0, 0, 1.05], [0, 0, 0, 1])
        first_state = [0]*len(self.mtrIdx)
        utils.resetCoordinates(1, self.mtrIdx, first_state)
        return first_state
    
    def step(action):
        next_state = clamp(action)
        utils.resetCoordinates(1, self.mtrIdx, next_state)
        reward = 0
        game_over = True
        additional_info = ''
        return next_state, reward, game_over, additional_info

ModuleNotFoundError: No module named '_utils_v2'

In [ ]:
motor_indexes = [29, 35, 37] # , 42,48,50

angles = [-30,30,0] # , 30, 0,-30

utils.resetCoordinates(1, motor_indexes, angles)

In [ ]:
mF.stop_enviroment()

In [2]:
import torch

class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.hw = 'Hello, World!'
        self.Linear_1 = torch.nn.Linear(26 ,  52)
        self.Linear_2 = torch.nn.Linear(52 , 104)
        self.Linear_3 = torch.nn.Linear(104, 208)
        self.Linear_4 = torch.nn.Linear(208, 104)
        self.Linear_5 = torch.nn.Linear(104,  52)
        self.Linear_6 = torch.nn.Linear(52 ,  26)
        
    def forward(self, x):
        x = self.Linear_1(x)
        x = self.Linear_2(x)
        x = self.Linear_3(x)
        x = self.Linear_4(x)
        x = self.Linear_5(x)
        x = self.Linear_6(x)
        return x 
    
def optimizer_init(net, lr  = 1e-3,  momentum = 0.9, 
                   wght_dcy = 5e-4,  epsilon  = 1e-8, beta1 = 0.9,  beta2 = 0.999, 
                   amsgrad  = False, is_SGD   = True, custom_params = True):
    if custom_params:
        biases, not_biases = list(), list()
        for param_name, param in net.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        params = [{'params': biases, 'lr': 2*lr}, {'params': not_biases}]
    else:
        params = net.parameters()
    if is_SGD:
        optimizer = torch.optim.SGD(
            params       = params,
            lr           = lr,
            momentum     = momentum,
            weight_decay = wght_dcy)
    else:
        optimizer = torch.optim.Adam(
            params       = params, 
            lr           = lr, 
            betas        = (beta1,beta2), 
            eps          = epsilon,
            weight_decay = wght_dcy, 
            amsgrad      = amsgrad)
    return optimizer

In [ ]:
path = './cfgs'
enviroment = daughter(path)
enviroment.run_enviroment()

# Net initialization
net = model()
# Optimizer initialization 
optimizer = optimizer_init(net)

for epoch in range(100):
    while condition:
        action = net(action)
        enviroment.step(action)
        reward = 0 #reward_fn(action)
        optimizer.zero_grad()
        reward.backward()
        optimizer.step()
    enviroment.reset()
    
enviroment.stop_enviroment()